<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Elsevier_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Elsevier

In [1]:
import requests 
from io import BytesIO
import re 
import os

In [2]:
!pip install elsapy

In [2]:
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json

In [3]:
# create config.json
dict_api =    {
	    "apikey": "856d163d12a1f149d801ee44e166857e",
	    "insttoken": ""
    }

api_json = json.dumps(dict_api)
# print(api_json)  


my_file = r'config.json'
with open(my_file, 'w', encoding='utf-8') as file:
    file.write(api_json)


## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

## Initialize client
client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

In [4]:
#   retrieving all results
doc_srch = ElsSearch("rechargeable lithium metal",'scopus')
doc_srch.execute(client, get_all = False)
print ("doc_srch has", len(doc_srch.results), "results.")

doc_srch has 25 results.


In [29]:
# get scopus ID
scopus_id = []
for p in doc_srch.results:
  if p['openaccess']==1:
    scopus_id.append(p['dc:identifier'])

In [44]:
# get doi
doi = []
for p in doc_srch.results:
  if p['openaccess']=='1':
    doi.append(p['prism:doi'])

In [49]:
doc_srch.results[9]

{'@_fa': 'true',
 'affiliation': [{'@_fa': 'true',
   'affiliation-city': 'Jaipur',
   'affiliation-country': 'India',
   'affilname': 'Malaviya National Institute of Technology Jaipur'}],
 'article-number': '3999',
 'citedby-count': '0',
 'dc:creator': 'Bhardwaj U.',
 'dc:identifier': 'SCOPUS_ID:85125982200',
 'dc:title': 'High energy storage capabilities of CaCu<inf>3</inf>Ti<inf>4</inf>O<inf>12</inf> for paper-based zinc–air battery',
 'eid': '2-s2.0-85125982200',
 'freetoread': {'value': [{'$': 'all'},
   {'$': 'publisherfullgold'},
   {'$': 'repository'},
   {'$': 'repositoryvor'},
   {'$': 'repositoryam'}]},
 'freetoreadLabel': {'value': [{'$': 'All Open Access'},
   {'$': 'Gold'},
   {'$': 'Green'}]},
 'link': [{'@_fa': 'true',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85125982200',
   '@ref': 'self'},
  {'@_fa': 'true',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85125982200?field=author,affiliation',
   '@ref': 'author-affiliation'},

In [48]:
len(doi)

25

In [47]:
"""
Elsevier web-scraper. PLEASE use your own API key.
"""

# __author__ = "Shu Huang"
# __email__ = "sh2009@cam.ac.uk"

import json
import requests
import numpy as np

API_KEY = "856d163d12a1f149d801ee44e166857e"
query = "rechargeable lithium metal"
base_url = 'https://api.elsevier.com/content/search/scopus'

data = {"qs": query,
        "date": 2019,
        # "volume": 0,
        "display": {
            "show": 10,
            "offset": 0
        }}

headers = {'x-els-apikey': API_KEY,
           'Content-Type': 'application/json',
           'Accept': 'application/json'}


def get_response(url, data, headers):
    """
    Return the response from Elsevier
    :param url: <str> base_url
    :param data: <dict> data parameters
    :param headers: <dict> headers
    :return: response
    """
    response = requests.put(url, data=json.dumps(data), headers=headers)
    response = response.text.replace('false', 'False').replace('true', 'True')
    try:
        response = eval(response)
    except BaseException:
        print(response)
    return response


def get_doi(data, volume, year):
    """
    Get DOIs
    :param data: <dict> data parameters
    :param volume: <int> the volume index
    :param year: <int> the year index
    :return: <list> of <str> list of DOIs
    """
    dois = []
    data['volume'] = volume
    data["date"] = year
    response = get_response(base_url, data, headers)
    if 'resultsFound' in response.keys():
        n = int(np.ceil(response['resultsFound'] / 100))
    else:
        n = 60
    for offset in range(n + 1):
        data["display"]["offset"] = offset
        response = get_response(base_url, data, headers)
        if 'results' in response.keys():
            results = response['results']
            for result in results:
                if 'doi' in result:
                    dois.append(result['doi'])
    return dois


# def download_doi(doi):
#     """
#     Download the paper according to the DOI
#     :param doi: <str> DOI
#     """
#     with open(str(doi) + '.xml', 'w', encoding='utf-8') as f:
#         request_url = 'https://api.elsevier.com/content/article/scopus_id/{}?apiKey={}&httpAccept=text%2Fxml'.format(
#             doi, API_KEY)
#         text = requests.get(request_url).text
#         f.write(text)
#     return

In [31]:
def download_scopus(doi):
    """
    Download the paper according to the DOI
    :param doi: <str> DOI
    """
    with open(str(doi) + '.xml', 'w', encoding='utf-8') as f:
        request_url = 'https://api.elsevier.com/content/article/scopus_id/{}?apiKey={}&httpAccept=text%2Fxml'.format(
            doi, API_KEY)
        print(request_url)
        text = requests.get(request_url).text
        f.write(text)
    return

In [23]:
def download_doi(doi):
    """
    Download the paper according to the DOI
    :param doi: <str> DOI
    """
    with open("doi/" + str(doi) + '.xml', 'w', encoding='utf-8') as f:
        request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(
            doi, API_KEY)
        print(request_url)
        text = requests.get(request_url).text
        f.write(text)
    return

In [38]:
 request_url = 'https://api.elsevier.com/content/article/scopus_id/{}?apiKey={}'.format(
            '85122632679', API_KEY)
 
 print(request_url)
 # getting the error - "Resource not found"

https://api.elsevier.com/content/article/scopus_id/85122632679?apiKey=856d163d12a1f149d801ee44e166857e


In [37]:
 request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(
            '10.1038/s41598-022-08691-2', 	'dad1b7295c38089e9ab8129bb2b77563')
 
 print(request_url)
 # getting the error - "Resource not found"

https://api.elsevier.com/content/article/doi/10.1038/s41598-022-08691-2?apiKey=dad1b7295c38089e9ab8129bb2b77563&httpAccept=text%2Fxml


In [32]:
for id in scopus_id:
  print(id)
  download_scopus(id)

SCOPUS_ID:85126720450
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126720450?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126558210
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126558210?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126531853
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126531853?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126269582
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126269582?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126250517
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126250517?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126237860
https://api.elsevier.com/content/article/scopus_id/SCOPUS_ID:85126237860?apiKey=856d163d12a1f149d801ee44e166857e&httpAccept=text%2Fxml
SCOPUS_ID:85126224116
https://api.elsevier.com/content/art